### bertscore


In [27]:
import pandas as pd
import numpy as np
from evaluate import load
from glob import glob
from datasets import load_dataset
from tqdm import tqdm
bertscore = load("bertscore")

In [28]:
from glob import glob
result_all_list = glob('./predictions_all/*')
result_last_list = glob('./predictions_last/*')

In [42]:
# ref(고정)
file_path = '/home/uj-user/Yo/hybrid-ltm/data_eval/context-fms/MS_DG/test_ref.jsonl'
df = pd.read_json(file_path, lines=True, orient='records')
output_df = pd.DataFrame(df[['output','eval_indicator']])
output_df = output_df[output_df['eval_indicator'] == 'Memorability']
print(output_df.index)


Index([2, 5, 6, 7, 8, 11, 20, 29, 34, 38, 42, 46, 51, 57, 63, 64, 65, 71, 76,
       79],
      dtype='int64')


In [43]:
# 전체 세션
all_df = pd.DataFrame()
for result in result_all_list:
    print(result)
    df2 = pd.read_json(result, lines=True, orient='records')
    pred_df = pd.DataFrame(df2['text'])
    pred_df = pred_df.loc[output_df.index]
    # 정답 output
    ref_list = output_df['output'].tolist()
    # 전체세션으로 생성 output
    pred_list = pred_df['text'].tolist()
    
    results = bertscore.compute(predictions=pred_list, references=ref_list, model_type="distilbert-base-uncased")
    print(results)
    results = dict(list(results.items())[:-1])
    average_dict = {key: np.mean([float(val) for val in value]) for key, value in results.items()}
    
    new_df = pd.DataFrame(average_dict, index= [result])
    all_df = pd.concat([all_df, new_df])


./predictions_all/DialoGPT-large_MS_DG.json
{'precision': [0.7606056928634644, 0.7834993004798889, 0.7603680491447449, 0.7363882660865784, 0.7705181241035461, 0.7661105990409851, 0.7617810964584351, 0.800658106803894, 0.7388308644294739, 0.8045691847801208, 0.795420229434967, 0.7776427268981934, 0.8166095018386841, 0.7499232292175293, 0.7124677896499634, 0.7377598881721497, 0.6953292489051819, 0.7611576914787292, 0.7590395212173462, 0.7767895460128784, 0.7595375776290894, 0.6900046467781067, 0.7594613432884216, 0.7838946580886841, 0.746921181678772, 0.7531148195266724, 0.7486879825592041, 0.8119406700134277, 0.7576755285263062, 0.8507586717605591, 0.732589066028595, 0.8179283142089844, 0.7062174081802368, 0.7629908919334412, 0.779987096786499, 0.811394214630127, 0.7250546216964722, 0.7172468304634094, 0.7702194452285767, 0.7110958099365234, 0.7109110951423645, 0.7512747645378113, 0.8150924444198608, 0.8186601400375366, 0.6963834762573242, 0.7836765050888062, 0.7783921957015991, 0.74290

{'precision': [0.7793918251991272, 0.8204200863838196, 0.8065146207809448, 0.7521085739135742, 0.8138951659202576, 0.8075563907623291, 0.8266184329986572, 0.7613562345504761, 0.7847817540168762, 0.8098112940788269, 0.7448091506958008, 0.8103911876678467, 0.8444867134094238, 0.8362613320350647, 0.868180513381958, 0.809029221534729, 0.7828328609466553, 0.8247876167297363, 0.7274609208106995, 0.7154949903488159, 0.7814276218414307, 0.9172037839889526, 0.7409437894821167, 0.8310704827308655, 0.7120547294616699, 0.7979334592819214, 0.7978576421737671, 0.8506088256835938, 0.7273670434951782, 0.7967798113822937, 0.66994309425354, 0.8286960124969482, 0.7177356481552124, 0.7728480100631714, 0.8078504800796509, 0.7701554298400879, 0.7673874497413635, 0.742874026298523, 0.8122721910476685, 0.6961662173271179, 0.8303364515304565, 0.8276115655899048, 0.8062839508056641, 0.7503551244735718, 0.7593332529067993, 0.731200098991394, 0.7006087303161621, 0.7533540725708008, 0.8749850988388062, 0.831929743

In [44]:
all_df

,precision,recall,f1
./predictions_all/DialoGPT-large_MS_DG.json,0.755372,0.667678,0.708186
./predictions_all/rebot-generation_MS_DG.json,0.785315,0.755224,0.769715
./predictions_all/llama-7b_ALL_MS_DG.json,0.760424,0.737554,0.748414
./predictions_all/GODEL-v1_1-large-seq2seq_MS_DG.json,0.736453,0.684400,0.709073
./predictions_all/llama-7b_MS_DG_MS_DG.json,0.774733,0.761170,0.767241
./predictions_all/blenderbot-3B_MS_DG.json,0.677711,0.651971,0.663919


In [32]:
# 마지막 세션
last_df = pd.DataFrame()
for result in result_last_list:
    print(result)
    df2 = pd.read_json(result, lines=True, orient='records')
    last_output_df = pd.DataFrame(df2['text'])
    # 정답 output
    ref_list = output_df['output'].tolist()
    # 마지막세션으로 생성 output
    last_pred_list = last_output_df['text'].tolist()
    
    results = bertscore.compute(predictions=last_pred_list, references=ref_list, model_type="distilbert-base-uncased")
    print(results)
    results = dict(list(results.items())[:-1])
    average_dict = {key: np.mean([float(val) for val in value]) for key, value in results.items()}
    
    new_df = pd.DataFrame(average_dict, index= [result])
    last_df = pd.concat([last_df, new_df])

./predictions_last/DialoGPT-large_MS_DG.json
{'precision': [0.7171797752380371, 0.7457655668258667, 0.7603679895401001, 0.7343679070472717, 0.7673180103302002, 0.7595049738883972, 0.7306994795799255, 0.8012790679931641, 0.7388309836387634, 0.8244552612304688, 0.7050952315330505, 0.7160584330558777, 0.935728907585144, 0.770484447479248, 0.7124677896499634, 0.8268065452575684, 0.7644521594047546, 0.7570821642875671, 0.7401525974273682, 0.827078104019165, 0.7435382604598999, 0.8394264578819275, 0.7293685674667358, 0.7838946580886841, 0.7997657060623169, 0.7767184972763062, 0.7339609861373901, 0.8119406700134277, 0.7594159841537476, 0.8418156504631042, 0.7325891852378845, 0.7748255729675293, 0.7062174081802368, 0.7332085371017456, 0.7888081073760986, 0.869716227054596, 0.664638102054596, 0.7563990950584412, 0.7790818810462952, 0.7204898595809937, 0.6880910992622375, 0.7832747101783752, 0.7892180681228638, 0.7910937666893005, 0.73160719871521, 0.7413750290870667, 0.7443187236785889, 0.74365

{'precision': [0.7917152643203735, 0.7809430360794067, 0.8162790536880493, 0.7340267300605774, 0.7559514045715332, 0.7581022381782532, 0.8048485517501831, 0.7906761169433594, 0.7011787295341492, 0.7561472654342651, 0.7183301448822021, 0.7779468297958374, 0.7278059124946594, 0.7031216621398926, 0.7625274658203125, 0.8124709129333496, 0.6786761283874512, 0.7708383202552795, 0.7492182850837708, 0.6723837852478027, 0.7931343913078308, 0.7720877528190613, 0.7018133401870728, 0.7787761688232422, 0.6841201782226562, 0.8472146987915039, 0.7367749810218811, 0.7183924913406372, 0.7224929928779602, 0.7691704630851746, 0.7093026041984558, 0.7436316013336182, 0.7093945741653442, 0.7641838192939758, 0.7103732824325562, 0.7709434032440186, 0.7539703249931335, 0.7484130859375, 0.6920777559280396, 0.7210074663162231, 0.750229001045227, 0.7339732646942139, 0.771918535232544, 0.6836838126182556, 0.7174056768417358, 0.6910680532455444, 0.7712395191192627, 0.0, 0.8183116316795349, 0.7433705925941467, 0.795

In [33]:
last_df

,precision,recall,f1
./predictions_last/DialoGPT-large_MS_DG.json,0.757679,0.665148,0.707883
./predictions_last/rebot-generation_MS_DG.json,0.772095,0.738632,0.754674
./predictions_last/GODEL-v1_1-large-seq2seq_MS_DG.json,0.743074,0.697021,0.718923
./predictions_last/llama-7b_ALL_MS_DG_last.json,0.730631,0.711415,0.720648
./predictions_last/blenderbot-3B_MS_DG.json,0.679908,0.642583,0.659838
./predictions_last/llama-7b_MS_DG_MS_DG_last.json,0.775872,0.762922,0.768963


In [34]:
final_df = pd.concat([all_df, last_df])
final_df

,precision,recall,f1
./predictions_all/DialoGPT-large_MS_DG.json,0.755372,0.667678,0.708186
./predictions_all/rebot-generation_MS_DG.json,0.785315,0.755224,0.769715
./predictions_all/llama-7b_ALL_MS_DG.json,0.760424,0.737554,0.748414
./predictions_all/GODEL-v1_1-large-seq2seq_MS_DG.json,0.736453,0.684400,0.709073
./predictions_all/llama-7b_MS_DG_MS_DG.json,0.774733,0.761170,0.767241
./predictions_all/blenderbot-3B_MS_DG.json,0.677711,0.651971,0.663919
./predictions_last/DialoGPT-large_MS_DG.json,0.757679,0.665148,0.707883
./predictions_last/rebot-generation_MS_DG.json,0.772095,0.738632,0.754674
./predictions_last/GODEL-v1_1-large-seq2seq_MS_DG.json,0.743074,0.697021,0.718923
./predictions_last/llama-7b_ALL_MS_DG_last.json,0.730631,0.711415,0.720648


In [36]:
## LAST, ALL Session bertscore 차이
empty_li = []
df = pd.DataFrame(empty_li)
df['DialoGPT'] = final_df.iloc[0] - final_df.iloc[6]
df['rebot'] = final_df.iloc[1] - final_df.iloc[7]
df['llama7b_all'] = final_df.iloc[2] - final_df.iloc[9]
df['GODEL'] = final_df.iloc[3] - final_df.iloc[8]
df['llama7b_MS_DG'] = final_df.iloc[4] - final_df.iloc[11]
df['blenderbot'] = final_df.iloc[5] - final_df.iloc[10]

df

,DialoGPT,rebot,llama7b_all,GODEL,llama7b_MS_DG,blenderbot
precision,-0.002307,0.013220,0.029794,-0.006621,-0.001139,-0.002197
recall,0.002531,0.016592,0.026139,-0.012621,-0.001752,0.009388
f1,0.000303,0.015041,0.027766,-0.009850,-0.001722,0.004081
